<a href="https://colab.research.google.com/github/jellyjel/woni.ai/blob/master/disclosure_warning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

투자주의/투자경고/투자위험 지정예고종목 등

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
 
# 1) reqeusts 라이브러리로 HTML 페이지 요청 
# 1-1) response.content로 데이터를 추출

headers = {
    'Connection': 'keep-alive',
    'Accept': 'text/html, */*; q=0.01',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'en-US,en;q=0.9',
}
 
data = {
  'method': 'searchDetailsSub',
  'currentPageSize': '100',
  'pageIndex': '1',
  'orderMode': '1',
  'orderStat': 'D',
  'forward': 'details_sub',
  'disclosureType01': '',
  'disclosureType02': '0342|0343|0341|',
  'disclosureType03': '',
  'disclosureType04': '',
  'disclosureType05': '',
  'disclosureType06': '',
  'disclosureType07': '',
  'disclosureType08': '',
  'disclosureType09': '',
  'disclosureType10': '',
  'disclosureType11': '',
  'disclosureType13': '',
  'disclosureType14': '',
  'disclosureType20': '',
  'pDisclosureType01': '',
  'pDisclosureType02': '0342|0343|0341|',
  'pDisclosureType03': '',
  'pDisclosureType04': '',
  'pDisclosureType05': '',
  'pDisclosureType06': '',
  'pDisclosureType07': '',
  'pDisclosureType08': '',
  'pDisclosureType09': '',
  'pDisclosureType10': '',
  'pDisclosureType11': '',
  'pDisclosureType13': '',
  'pDisclosureType14': '',
  'pDisclosureType20': '',
  'searchCodeType': '',
  'repIsuSrtCd': '',
  'allRepIsuSrtCd': '',
  'oldSearchCorpName': '',
  'disclosureType': '',
  'disTypevalue': '',
  'reportNm': '지정예고',
  'reportCd': '',
  'searchCorpName': '',
  'business': '',
  'marketType': '',
  'settlementMonth': '',
  'securities': '',
  'submitOblgNm': '',
  'enterprise': '',
  'fromDate': '2020-01-17',
  'toDate': '2020-02-17',
  'reportNmTemp': '',
  'reportNmPop': '',
  'disclosureTypeArr02': '0342',
  'disclosureTypeArr02': '0343',
  'disclosureTypeArr02': '0341',
  'bfrDsclsType': 'on'
}
 
response = requests.post('http://kind.krx.co.kr/disclosure/details.do', headers=headers, data=data, verify=False)

In [0]:
print(response.text[:100]) # 앞부분만 출력

In [0]:
# 2) HTML 페이지 파싱 BeautifulSoup(HTML데이터, 파싱방법)
# 2-1) BeautifulSoup 파싱방법
soup = BeautifulSoup(response.content, 'html.parser')
 
# 3) 필요한 데이터 검색
#alert = soup.find(id="main-contents")
# 4) 필요한 데이터 추출

stock_idx= soup.select('.txc')
stock_idx[:5]

**인덱스와 지정예정일 뽑기**

In [0]:
date = soup.select(".txc")
date_text = [item.get_text().strip() for item in date ]
len(date_text)
c1 = []
c2 = []

for idx, val in enumerate(date_text):
   # print(idx, val)
    if (int(idx) % 3 == 0):
      c1.append(val)
    elif(int(idx) % 3 == 1):
      c2.append(val)
    else:
      continue

df = pd.DataFrame({'index':c1,'time':c2})
df

In [0]:
elements = soup.find_all('a')
elements[:5]

**종목코드**

https://stackoverflow.com/questions/23380171/using-beautifulsoup-to-extract-text-without-tags

In [0]:
# Find all <a> elements
elements = soup.find_all('a')
code=[]
# Loop over all found elements
for element in elements:
    # Disregard element if it doesn't contain onclick attribute
    if 'onclick' not in element.attrs:
        continue
    # Get attribute value
    value = element['onclick']
    # Disregard wrong elements
    if not value.startswith('openDisclsChart('):
        continue
    # Extract position of opening bracket
    position = value.index('(') + 1
    # Slice string so only content inside bracket is left
    value = value[position:-15]
    code.append(value)
    # Print result
    print(value)


In [0]:
len(code)

In [0]:
# Find all <a> elements
elements = soup.find_all('a')

# Loop over all found elements
for element in elements:
    # Disregard element if it doesn't contain onclick attribute
    if ' title' not in element.attrs:
        continue
    # Get attribute value
    value = element[' title']
    # Disregard wrong elements
    # Extract position of opening bracket
    position = value.index('"') + 1
    # Slice string so only content inside bracket is left
    value = value[position:-1]
    # Print result
    print(value)

In [0]:
<img src="/images/common/icn_t_yu.gif" class="vmiddle legend" alt="유가증권">

In [0]:
stock_title = [item.get_text().strip() for item in elements]
#name = stock_name[7:13]
stock_title
c1 = []
c2 = []

for idx, val in enumerate(stock_title):
   # print(idx, val)
    if (int(idx) % 4 == 0):
      c1.append(val)
    elif(int(idx) % 4 == 1):
      c2.append(val)
    else:
      continue

#노이즈/특수기호 제거
df2 = pd.DataFrame({'stockname':c1[:-2],'code':code,'title':c2[:-2]})
df2['code'] = df2['code'].str.replace(pat=r'[^A-Za-z0-9]', repl= r' ', regex=True)

dis_warning_df= df.join(df2)
dis_warning_df.head
dis_warning_df.to_csv('시장조치 공시예고'+ '.csv', index=False)

In [0]:
def create_custom_table(stocks, links):
  table = []
  for idx, item in enumerate(stocks):
    stocks = 
